### Cifar10 convolutional neural network

Loading and preparing data

In [10]:
import tensorflow as tf
import numpy as np

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()


In [8]:
print(f"Dataset training size X={X_train.shape}, Y={y_train.shape}, validation size X={X_test.size}, Y={y_test.shape}")

Dataset training size X=(50000, 32, 32, 3), Y=(50000, 1), validation size X=30720000, Y=(10000, 1)


In [9]:
X_train_scaled = X_train / 255
X_test_scaled =  X_test / 255

In [12]:
print(f"Unique output are: {np.unique(y_train)}, where every number represents different class")

Unique output are: [0 1 2 3 4 5 6 7 8 9], where every number represents different class


In [13]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=10, dtype=np.float32)
y_test_one_hot = to_categorical(y_test, num_classes=10, dtype=np.float32)

In [29]:
from keras.layers import Flatten, Layer, Conv2D, MaxPooling2D, Dense
from keras import Sequential

with tf.device('/GPU:0'):
    simple_conv_model = Sequential([
        Conv2D(32, (5,5), activation='relu'),
        MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
        Conv2D(64, (5,5), strides=(1,1)),
        MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
        Flatten(),
        Dense(400, activation='relu'),
        Dense(120, activation='relu'),
        Dense(40, activation='relu'),
        Dense(10, activation='softmax')
    ])

    simple_conv_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    simple_conv_model.fit(X_train_scaled, y_train_one_hot, batch_size=256, validation_data=(X_test_scaled, y_test_one_hot), epochs=100)

Epoch 1/100
196/196 [==============================] - 2s 8ms/step - loss: 1.8633 - accuracy: 0.3251 - val_loss: 1.6051 - val_accuracy: 0.4196
Epoch 2/100
196/196 [==============================] - 1s 6ms/step - loss: 1.4503 - accuracy: 0.4860 - val_loss: 1.3957 - val_accuracy: 0.5044
Epoch 3/100
196/196 [==============================] - 1s 6ms/step - loss: 1.2350 - accuracy: 0.5661 - val_loss: 1.2809 - val_accuracy: 0.5535
Epoch 4/100
196/196 [==============================] - 1s 6ms/step - loss: 1.0860 - accuracy: 0.6223 - val_loss: 1.2649 - val_accuracy: 0.5699
Epoch 5/100
196/196 [==============================] - 1s 6ms/step - loss: 0.9616 - accuracy: 0.6648 - val_loss: 1.1280 - val_accuracy: 0.6093
Epoch 6/100
196/196 [==============================] - 1s 6ms/step - loss: 0.8548 - accuracy: 0.7026 - val_loss: 1.0376 - val_accuracy: 0.6370
Epoch 7/100
196/196 [==============================] - 1s 6ms/step - loss: 0.7575 - accuracy: 0.7356 - val_loss: 1.1118 - val_accuracy: 0.6207

First model evaluation is showing ENORMOUS overfitting after 100 epochs loss on train set is 0.03 wheras on validation set it is over 5. To prevent this we are going to apply dropout regularization and create EarlyStopping instance to stop learning after certain point

In [45]:
from keras.layers import Dropout

simple_conv_model_v2 = Sequential([
    Conv2D(32, (5,5)),
    MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    Conv2D(64, (5,5), strides=(1,1)),
    MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'),
    Flatten(),
    Dense(400, activation='relu'),
    Dropout(0.5),
    Dense(120, activation='relu'),
    Dropout(0.5),
    Dense(40, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

simple_conv_model_v2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_loss') - logs.get('loss') > 1.:
            print("\nDifference between val_loss and loss is bigger than 1!")
            self.model.stop_training = True
        if logs.get('val_accuracy') - logs.get('accuracy') < -0.05:
            print("\nDifference between val_accuracy and train accuracy is bigger than -0.05!")
            self.model.stop_training = True
callback = MyCallback()

In [47]:
simple_conv_model_v2.build(X_train_scaled.shape)
simple_conv_model_v2.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (50000, 28, 28, 32)       2432      
                                                                 
 max_pooling2d_16 (MaxPoolin  (50000, 14, 14, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (50000, 10, 10, 64)       51264     
                                                                 
 max_pooling2d_17 (MaxPoolin  (50000, 5, 5, 64)        0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (50000, 1600)             0         
                                                                 
 dense_32 (Dense)            (50000, 400)             

In [48]:
simple_conv_model_v2.fit(X_train_scaled,
                        y_train_one_hot,
                        epochs=100,
                        batch_size=512,
                        callbacks=[callback],
                        validation_data=(X_test_scaled, y_test_one_hot),
                        validation_batch_size=512)

Epoch 1/100
98/98 [==============================] - 2s 12ms/step - loss: 2.0880 - accuracy: 0.2286 - val_loss: 1.7311 - val_accuracy: 0.3941
Epoch 2/100
98/98 [==============================] - 1s 9ms/step - loss: 1.7509 - accuracy: 0.3657 - val_loss: 1.4656 - val_accuracy: 0.4896
Epoch 3/100
98/98 [==============================] - 1s 9ms/step - loss: 1.6052 - accuracy: 0.4253 - val_loss: 1.3772 - val_accuracy: 0.5273
Epoch 4/100
98/98 [==============================] - 1s 9ms/step - loss: 1.4829 - accuracy: 0.4782 - val_loss: 1.2765 - val_accuracy: 0.5535
Epoch 5/100
98/98 [==============================] - 1s 9ms/step - loss: 1.3924 - accuracy: 0.5146 - val_loss: 1.2063 - val_accuracy: 0.5840
Epoch 6/100
98/98 [==============================] - 1s 10ms/step - loss: 1.3296 - accuracy: 0.5393 - val_loss: 1.1568 - val_accuracy: 0.6021
Epoch 7/100
98/98 [==============================] - 1s 9ms/step - loss: 1.2767 - accuracy: 0.5604 - val_loss: 1.1477 - val_accuracy: 0.5949
Epoch 8/100